# GRADED CHALLENGE 3

<h3>MUHAMAD NATUAL HISAK</h3>

Kamu adalah seorang data analis di The Look yang merupakan salah satu platform e-commerce terbesar di planet Mars. Kamu diminta untuk membuat laporan evaluasi aktivitas penjualan di platform tersebut.

Untuk mempermudah pekerjaan kamu supaya terarah, kamu harus menentukan problem statement dengan success criteria berdasarkan SMART. Namun tantangannya, kamu tentukan problem statement berdasarkan penjabaran-penjabaran analisis dalam bentuk persoalan yang harus kamu jawab menggunakan Query SQL. (SMART akan berdasarkan dari poin-poin penjabaran).

Catatan: Tidak perlu membuat plot/visualisasi data. Cukup tampilkan dataframe!

Poin penjabaran:

1. Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?

2. Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete. Apa informasi yang bisa kamu sampaikan?

3. Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?

4. Kategori produk apa saja yang paling banyak dibeli (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

5. Kategori produk apa saja yang paling banyak mendapatkan pendapatan (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

6. Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan? (Hint: Kamu bisa menggunakan korelasi)

<h4>POIN ANALISIS</h4>

1. Berikan kesimpulan dari laporan/informasi yang dibuat berdasarkan problem statement/poin penjabaran menggunakan bahasa awam! (boleh dalam beberapa paragraf)

<h4>POIN PERTANYAAN</h4>

1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!

2. Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?

3. CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!

# WORKING AREA

<h1>Problem Statement</h1>

**Mengetahui kategori dengan pendapatan terbesar tiap bulan pada kuartal 1 sampai kuartal 3 di tahun 2022**

Import library pandas dan hubungkan google colab dengan bigquery 

In [1]:
import pandas as pd

In [2]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "natual-try" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


1. Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?

In [3]:
df1 = client.query('''
SELECT  extract(year from created_at) as year,
        extract(quarter from created_at) as quarter,
        extract(month from created_at) as month, 
        count(order_id) as Total_Transaction

FROM `bigquery-public-data.thelook_ecommerce.orders`
WHERE status='Complete' and created_at>="2022-01-01" and created_at<"2022-10-01"
GROUP BY year, quarter,month
ORDER BY year, quarter,month ASC
''').to_dataframe()

In [4]:
df1

,year,quarter,month,Total_Transaction
0,2022,1,1,1087
1,2022,1,2,1007
2,2022,1,3,1303
3,2022,2,4,1387
4,2022,2,5,1478
5,2022,2,6,1543
6,2022,3,7,1898
7,2022,3,8,2177
8,2022,3,9,2443


Dari tabel diatas bisa dilihat bahwa jumlah transaksi yang berstatus "complete" tiap bulannya hampir selalu bertambah (kecuali bulan 1 ke bulan 2 ) dengan jumlah yang signifikan

2. Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete. Apa informasi yang bisa kamu sampaikan?

In [5]:
df2 = client.query('''
SELECT year, quarter, month, sum(qty*price) as total, sum(qty) as qty from ( SELECT EXTRACT(year FROM a.created_at) AS year,
EXTRACT(month FROM a.created_at) AS month,
extract(quarter from a.created_at) as quarter,
num_of_item as qty,
sale_price as price
FROM bigquery-public-data.thelook_ecommerce.orders a
JOIN bigquery-public-data.thelook_ecommerce.order_items b
ON a.order_id = b.order_id
WHERE a.status='Complete' and a.created_at>"2022-01-01" and a.created_at<"2022-10-01"
GROUP BY year, quarter, month, qty, price
ORDER BY year, quarter, month, qty, price ASC)
GROUP BY year, quarter, month
ORDER BY year, quarter, month
''').to_dataframe()

In [6]:
df2

,year,quarter,month,total,qty
0,2022,1,1,125526.790053,2030
1,2022,1,2,123660.010043,1894
2,2022,1,3,154831.939912,2346
3,2022,2,4,154638.760116,2352
4,2022,2,5,166650.109866,2574
5,2022,2,6,172714.050059,2540
6,2022,3,7,198285.379987,2936
7,2022,3,8,235673.170093,3434
8,2022,3,9,253442.819841,3703


Mirip dengan tabel pertama, total penjualan tiap bulan dari Q1-Q3 tahun 2022 juga hampir selalu mengalami peningkatan kecuali pada periode bulan 1 ke bulan 2 dan bulan 3 ke bulan 4.

3. Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?

In [7]:
df3 = client.query('''
SELECT DISTINCT  extract(year from created_at) as year,
        extract(quarter from created_at) as quarter,
        extract(month from created_at) as month, 
        Count(user_id) as Total_User
        
FROM `bigquery-public-data.thelook_ecommerce.orders`
WHERE status='Complete' and created_at>="2022-01-01" and created_at<"2022-10-01"
GROUP BY year, quarter,month
ORDER BY year, quarter,month ASC
''').to_dataframe()

In [8]:
df3

,year,quarter,month,Total_User
0,2022,1,1,1087
1,2022,1,2,1007
2,2022,1,3,1303
3,2022,2,4,1387
4,2022,2,5,1478
5,2022,2,6,1543
6,2022,3,7,1898
7,2022,3,8,2177
8,2022,3,9,2443


Dari data diatas bisa dilihat bahwa total user yang melakukan transaksi berstatus complete juga selalu bertambah dari bulan ke bulan kecuali pada bulan 1 ke bulan 2

4. Kategori produk apa saja yang paling banyak dibeli (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [9]:
df4 = client.query('''
SELECT * EXCEPT(ranks)
FROM
(
        SELECT year, quarter, month, category, 
        RANK() OVER(PARTITION BY year, quarter, month ORDER BY sum_of_item DESC) as ranks, 
        sum_of_item
        FROM(
                SELECT category,
                        extract(year from a.created_at) as year,
                        extract(quarter from a.created_at) as quarter,
                        extract(month from a.created_at) as month,
                        SUM(num_of_item) sum_of_item
                FROM bigquery-public-data.thelook_ecommerce.order_items a
                LEFT JOIN bigquery-public-data.thelook_ecommerce.products b ON a.product_id = b.id
                LEFT JOIN bigquery-public-data.thelook_ecommerce.orders c ON c.order_id = a.order_id
                WHERE a.status='Complete' 
                AND a.created_at>="2022-01-01" 
                AND a.created_at<"2022-10-01"
                GROUP BY 1,2,3,4
                ORDER BY year, quarter,month, category ASC
        )
        ORDER BY 1,2,3,5
)
WHERE ranks = 1 
ORDER BY 1,2,3
''').to_dataframe()

In [10]:
df4

,year,quarter,month,category,sum_of_item
0,2022,1,1,Intimates,227
1,2022,1,2,Intimates,219
2,2022,1,3,Jeans,268
3,2022,2,4,Intimates,277
4,2022,2,5,Intimates,313
5,2022,2,6,Intimates,330
6,2022,3,7,Jeans,374
7,2022,3,8,Intimates,456
8,2022,3,9,Intimates,528


Kategori yang plaing banyak dibeli tiap bulannya didominasi oleh kategori Intimates dengan 7x menduduki posisi teratas dan 2x lainnya ada pada kategori jeans. Total penjualan juga terlihat selalu meningkat tiap bulannya walaupun ada perbedaan pada kategori teratas.

5. Kategori produk apa saja yang paling banyak mendapatkan pendapatan (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [11]:
df5 = client.query('''
SELECT * EXCEPT(ranks)
FROM(
SELECT  year,quarter, month,
        category,
        SUM(gross_income_per_product) total_sales_per_category,
        RANK() OVER(PARTITION BY year,quarter, month ORDER BY SUM(gross_income_per_product) desc) ranks
FROM(
SELECT extract(year from a.created_at) as year,
       extract(quarter from a.created_at) as quarter,
       extract(month from a.created_at) as month,
       category, 
       product_id, 
       SUM(num_of_item) 
       OVER(PARTITION BY product_id, 
                        extract(year from a.created_at),
                        extract(quarter from a.created_at),
                        extract(month from a.created_at)) * (retail_price-cost) as gross_income_per_product
FROM bigquery-public-data.thelook_ecommerce.order_items a 
LEFT JOIN bigquery-public-data.thelook_ecommerce.products b ON a.product_id = b.id
LEFT JOIN bigquery-public-data.thelook_ecommerce.orders c ON c.order_id = a.order_id
WHERE a.status='Complete' AND a.created_at>="2022-01-01" AND a.created_at<"2022-10-01" )
GROUP BY 1,2,3,4)
WHERE ranks = 1
ORDER BY 1,2,3
''').to_dataframe()

In [12]:
df5

,year,quarter,month,category,total_sales_per_category
0,2022,1,1,Outerwear & Coats,10765.534000
1,2022,1,2,Outerwear & Coats,12073.731076
2,2022,1,3,Jeans,15037.016029
3,2022,2,4,Outerwear & Coats,15642.285051
4,2022,2,5,Outerwear & Coats,16202.729684
5,2022,2,6,Jeans,15654.971919
6,2022,3,7,Outerwear & Coats,29165.333734
7,2022,3,8,Outerwear & Coats,24396.153684
8,2022,3,9,Outerwear & Coats,41191.036824


Berbeda dengan data sebelumnya, kategori dengan total pendapatan yang terbesar bukanlah 2 kategori yang menjadi 7x top seller tiap bulannya (intimates), tetapi outerwear&coats dan Jeans. pendapatan per kategori inijuga selalu bertambah tiap bulannya.

6. Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan? (Hint: Kamu bisa menggunakan korelasi)

In [13]:
df6 = client.query('''
SELECT * EXCEPT(ranks,year,quarter,month)
FROM(
SELECT  year,quarter, month,
        SUM(gross_income_per_product) total_sales_per_category,
        COUNT(user_id) total_user,
        COUNT(order_id) total_order,
        RANK() OVER(PARTITION BY year,quarter, month ORDER BY SUM(gross_income_per_product) DESC) ranks
FROM(
SELECT extract(year from a.created_at) as year,
       extract(quarter from a.created_at) as quarter,
       extract(month from a.created_at) as month,
       category, 
       product_id,
       a.user_id, 
       a.order_id,
       SUM(num_of_item) 
       OVER(PARTITION BY product_id, 
                        extract(year from a.created_at),
                        extract(quarter from a.created_at),
                        extract(month from a.created_at)) * (retail_price-cost) as gross_income_per_product
FROM bigquery-public-data.thelook_ecommerce.order_items a 
LEFT JOIN bigquery-public-data.thelook_ecommerce.products b ON a.product_id = b.id
LEFT JOIN bigquery-public-data.thelook_ecommerce.orders c ON c.order_id = a.order_id
WHERE a.status='Complete' AND a.created_at>="2022-01-01" AND a.created_at<"2022-10-01" )
GROUP BY 1,2,3)
WHERE ranks = 1
ORDER BY 1,2,3
''').to_dataframe()

In [14]:
df6

,total_sales_per_category,total_user,total_order
0,85252.195518,1458,1458
1,91709.186361,1555,1555
2,120709.269672,1896,1896
3,125832.826280,2002,2002
4,132686.722724,2142,2142
5,138026.857249,2202,2202
6,179723.238328,2711,2711
7,201982.345508,3146,3146
8,238576.101699,3536,3536


In [15]:
df6.corr().nlargest(38, 'total_user')['total_user']

total_user                  1.00000
total_order                 1.00000
total_sales_per_category    0.99824
Name: total_user, dtype: float64

Tentu saja dari nilai korelasi diatas, jumlah user yang melakukan transaksi lebih berkaitan dengan jumlah transaksi yang terjadi karena tiap transaksi pasti dilakukan oleh user, sehingga perbandingannya juga pasti akan 1:1

<h4>POIN ANALISIS</h4>

1. Berikan kesimpulan dari laporan/informasi yang dibuat berdasarkan problem statement/poin penjabaran menggunakan bahasa awam! (boleh dalam beberapa paragraf)


Dari poin poin penjabaran diatas, bisa disimpulkan bahwa :
1. Total transaksi tiap bulannya dari Q1-Q3 tahun 2022 selalu bertambah kecuali bulan 1 ke bulan 2

2. Total Penjualan (dalam USD) tiap bulannya dari Q1-Q3 tahun 2022 selalu bertambah kecuali bulan 1 ke bulan 2

3. Jumlah user yang melakukan transaksi berstatus complete juga selalu bertambah tiap bulan, tetapi angkanya berada dibawah total transaksi. Hal ini menandakan bahwa ada user yang melakukan lebih dari 1x transaksi.

4. Kategori yang paling banyak dibeli tiap bulannya tidak bervariasi, hanya ada 2 kategori yang selalu menjadi top seller yaitu Intimates (sebanyak 7x) dan Jeans(sebanyak 2x)

5. Kategori yang paling banyak berkontribusi dalam pendapatan perbulan ternyata bukan berasal dari kategori top seller (intimates) melainkan dari kategori Outerwear&Coats dan Jeans

6. jumlah user yang melakukan transaksi lebih berkaitan dengan jumlah transaksi yang terjadi karena tiap transaksi pasti dilakukan oleh user, sehingga perbandingannya juga pasti akan 1:1

<h4>POIN PERTANYAAN</h4>

1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!

2. Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?

3. CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!


<h4>Jawaban</h4>

1. Dapat terukur, karena bisa didapat nilai kategori dengan pendapatan terbesar tiap bulan pada Q1-Q3 tahun 2022. 

2. Sangat masuk akal karena jika dilihat trend pendapatan tiap bulan, selalu bertambah dan pada akhir kuartal 3 tahun 2022, pendapatan sudah mencapai angka 253442.81 USD yang berarti targetnya sekarang adalah mempertahankan angka tersebut agar tidak turun dibawah 250000 USD pada kuartal 4

3. Dari trend transaksi dan jumlah transaksi pada 3 kuartal terakhir, angkanya selalu meningkat dengan jumlah transaksi yang lebih banyak dari jumlah user. itu artinya ada user yang melakukan lebih dari 1x transaksi. Untuk GMV bisa dihitung dengan jumlah item terjual * pendapatan. didapatkan gmv sebesar 938.498.762,46. Dengan penjabaran diatas, menurut saya perusahaan ini sangat layak untuk mendapat investor baru agar perkebangan penjualan bisa lebih cepat dan keuntungan yang diraup meningkat